# Prepare Workloads

In this notebook, we will try to prepare the workloads that we will be applying
to the deployments. Potentially, this leads to functions that will be called
when we want to send traffic to the deployment and we want to measure the system's
performance.

In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import itertools

import os
import time

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

fetching imagenet v2
resizing images


  0%|          | 0/100 [00:00<?, ?it/s]

converting to bentoml files


  0%|          | 0/100 [00:00<?, ?it/s]

extracting base64 files


  0%|          | 0/100 [00:00<?, ?it/s]

preprocessing for mobilenet


  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
def experiment_batch(warmup_req_count, total_req_count, batch_size, service_name, ):
    # an array as the same length as 
    batch_results = {
        'response_times_ms': [],
    }
    reqs_failed = 0
    reqs_succeeded = 0

    print('warming up...')
    for _ in range(warmup_req_count):
        try:
            # discard the results
            request_funcs.workload_funcs[service_name](batch_size=batch_size)
        except Exception:
            print('exception occured:')
            traceback.print_exc()

    # running the main workload
    print(f'running {service_name} workload function, batch_size: {batch_size}')
    for _ in tqdm(range(total_req_count)):
        try:
            result = request_funcs.workload_funcs[service_name](batch_size=batch_size)
            batch_results['response_times_ms'].append(result['response_time_ms'])
            reqs_succeeded += 1
        except Exception:
            print('exception occured:')
            traceback.print_exc()
            reqs_failed += 1

    batch_results.update({
        'reqs_failed': reqs_failed,
        'reqs_succeeded': reqs_succeeded,
    })

    return batch_results

def perform_experiment_batch(config):
    # configurables = ['batch_size']
    configurables = [k for k in config if isinstance(config[k], list)]
    print(f'configurables: {configurables}')

    # make a copy of config
    config_base = {k:config[k] for k in config if k not in configurables}
    configurable_base = {k:config[k] for k in config if k in configurables}

    results = []

    config_combinations_keys = list(configurable_base.keys())
    for config_combination in itertools.product(*configurable_base.values()):
        configurable = {config_combinations_keys[i]: config_combination[i]  for i in range(len(config_combinations_keys))}
        
        print(configurable)
        new_config = {**config_base}
        new_config.update(configurable)

        result = experiment_batch(**new_config)
        new_config.update(result)
        results.append(new_config)
        
    return results

config = {
    'warmup_req_count': 20,
    'total_req_count': 100,
    'service_name': 'bentoml-iris',
    'batch_size': [1,5,10,20,50,100],
}
results = perform_experiment_batch(config)

configurables: ['batch_size']
{'batch_size': 1}
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 5}
warming up...
running bentoml-iris workload function, batch_size: 5


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 10}
warming up...
running bentoml-iris workload function, batch_size: 10


...

  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 50}
warming up...
running bentoml-iris workload function, batch_size: 50


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 100}
warming up...
running bentoml-iris workload function, batch_size: 100


  0%|          | 0/100 [00:00<?, ?it/s]

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def post_process(df):
    df['resp_time_avg'] = df.apply(lambda x: np.mean(x['response_times_ms']), axis=1)
    for percentile in [50,90,95,99]:
        df[f'resp_time_p{percentile}'] = df.apply(lambda x: np.percentile(x['response_times_ms'], percentile), axis=1)
    return df


df = pd.DataFrame(data=results)
df = post_process(df)
df

,warmup_req_count,total_req_count,service_name,batch_size,response_times_ms,reqs_failed,reqs_succeeded,resp_time_avg,resp_time_p50,resp_time_p90,resp_time_p95,resp_time_p99
0,20,100,bentoml-iris,1,"[26.89, 18.044, 20.230999999999998, 17.253, 18...",0,100,19.96643,19.4410,23.3454,24.41720,28.08538
1,20,100,bentoml-iris,5,"[20.544, 20.009, 21.628999999999998, 17.489, 1...",0,100,20.88932,19.7870,23.8896,27.23205,34.07918
2,20,100,bentoml-iris,10,"[24.203, 23.233, 19.293000000000003, 20.333000...",0,100,20.74498,20.1935,23.8052,25.93745,30.95694
3,20,100,bentoml-iris,20,"[24.035, 20.756, 18.629, 18.707, 22.9290000000...",0,100,24.47792,20.4695,24.9953,35.47165,88.38873
4,20,100,bentoml-iris,50,"[35.085, 59.354, 35.26, 246.504, 251.726999999...",0,100,51.07665,35.8220,55.6829,244.94505,248.95401
5,20,100,bentoml-iris,100,"[43.67, 37.456, 45.902, 39.786, 249.002, 23.23...",0,100,75.57895,39.9425,246.3886,249.97240,272.32491


In [48]:
all_configs = [
    {
        'warmup_req_count': 200,
        'total_req_count': 1000,
        'service_name': 'bentoml-iris',
        'batch_size': [1,5,10,20,50,100],
    },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'tfserving-resnetv2',
        'batch_size': [1,2,3,5,10],
    },
    {
        'warmup_req_count': 20,
        'total_req_count': 1000,
        'service_name': 'tfserving-mobilenetv1',
        'batch_size': [1,2,3,5,10],
    },
]

results_folder = './results/batch_experiments_defult'
!mkdir -p {results_folder}
for config in all_configs:
    service_name = config['service_name']

    results = perform_experiment_batch(config)
    df = pd.DataFrame(data=results)

    now = util.get_time_with_tz()
    res_name = now.strftime(f'{service_name}_%Y-%m-%d_%H-%M-%S')
    print('res_name:', res_name)
    df.to_csv(f'{results_folder}/{res_name}.csv')

configurables: ['batch_size']
{'batch_size': 1}
warming up...
running bentoml-iris workload function, batch_size: 1


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 5}
warming up...
running bentoml-iris workload function, batch_size: 5


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 10}
warming up...
running bentoml-iris workload function, batch_size: 10


...

{'batch_size': 50}
warming up...
running bentoml-iris workload function, batch_size: 50


  0%|          | 0/100 [00:00<?, ?it/s]

{'batch_size': 100}
warming up...
running bentoml-iris workload function, batch_size: 100


  0%|          | 0/100 [00:00<?, ?it/s]

res_name: bentoml-iris_2021-05-04_23-13-34
